In [26]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import cv2
%matplotlib inline

In [27]:
import glob
#Data folders
bottle_dir ='E:\\HOG\\training_data\\bottle'
mug_dir = 'E:\\HOG\\training_data\\mug'
negative_dir = 'E:\\HOG\\training_data\\negative'
# images are divided up into vehicles and non-vehicles
bottle = []
mug = []
negative = []
# Read 
images = glob.iglob(bottle_dir + '/**/*.jpg', recursive=True)

for image in images:
        bottle.append(image)
        
images = glob.iglob(mug_dir + '/**/*.jpg', recursive=True)

for image in images:
        mug.append(image)

images = glob.iglob(negative_dir + '/**/*.jpg', recursive = True)

for image in images:
        negative.append(image)

In [28]:
from skimage.feature import hog

def get_hog_features(img, orient, pix_per_cell, cell_per_block, 
                        vis=False, feature_vec=True):
    """
    Return the hog features of the given input image
    Call with two outputs if vis==True"""
    if vis == True:
        features, hog_image = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), transform_sqrt=True, 
                                  visualise=vis, feature_vector=feature_vec)
        return features, hog_image
    # Otherwise call with one output
    else:      
        features = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), transform_sqrt=True, 
                       visualise=vis, feature_vector=feature_vec)
        return features

In [29]:
def extract_feature(image):
    features = []
    for file in image:
        file_features = []
        img = cv2.imread(file)
        reshape = cv2.resize(img,(100,100))
        feature_image = np.copy(reshape)
        hog_features = []
        hog_features.append(get_hog_features(feature_image ,9, 8, 2,vis=False, feature_vec=True))
        hog_features = np.ravel(hog_features)
        file_features.append(hog_features)
        features.append(np.concatenate(file_features))
    return features

In [30]:
import time
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler


In [31]:
bottle_features = extract_feature(bottle)
print(len(bottle_features))

c:\users\marzan shuvo\appdata\local\programs\python\python35\lib\site-packages\skimage\feature\_hog.py:150: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15. To supress this message specify explicitly the normalization method.
  skimage_deprecation)
c:\users\marzan shuvo\appdata\local\programs\python\python35\lib\site-packages\skimage\feature\_hog.py:248: skimage_deprecation: Argument `visualise` is deprecated and will be changed to `visualize` in v0.16
  'be changed to `visualize` in v0.16', skimage_deprecation)


1002


In [32]:
mug_features = extract_feature(mug)
print(len(mug_features))

1002


In [33]:
negative_features = extract_feature(negative)
print(len(negative_features))

1002


In [34]:
X = np.vstack((bottle_features,mug_features,negative_features)).astype(np.float64)


In [35]:
x = np.arange(len(negative_features), dtype=int)
y = np.hstack((np.zeros(len(bottle_features)), np.ones(len(mug_features)), np.full_like(x, 2)))
print(len(y))
print(y)

3006
[0. 0. 0. ... 2. 2. 2.]


In [36]:
gnb = GaussianNB()
gnb.fit(X, y)
svc = LinearSVC()
svc.fit(X, y)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [37]:
import pickle
filename = 'E:\\HOG\\finalized_model.sav'
pickle.dump(svc, open(filename, 'wb'))